# Predicción de victorias en Pokémon Showdown ([Gen 9] OU)

Este notebook reemplaza el dataset sintético anterior y utiliza **replays reales** descargados automáticamente desde Pokémon Showdown.

Seguiremos la rúbrica del curso: EDA completo, pipeline reproducible de preprocesamiento, comparación de 4 modelos (baseline + 2 vistos en clase + 1 moderno) y análisis con validación cruzada.

## 1. Carga de datos reales

Los datos provienen del script `scrape_showdown_replays.py`, que descarga replays recientes del formato `[Gen 9] OU`, extrae los equipos completos (6 Pokémon por jugador), las estadísticas base desde PokéAPI y etiqueta automáticamente quién ganó.

> Dependencias clave: `pandas`, `numpy`, `seaborn`, `matplotlib`, `scikit-learn`, `lightgbm`.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='whitegrid')

DATA_PATH = 'pokemon_showdown_teams_clean.csv'
df = pd.read_csv(DATA_PATH)
print(f'Filas: {len(df)} - Columnas: {df.shape[1]}')
df.head()

## 2. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Balance del target y valores faltantes clave
print(df['won_battle'].value_counts(normalize=True))
print('
Porcentaje con rating disponible:', (df[['player_rating','opponent_rating']].notna().all(axis=1).mean()*100).round(2))
print('
Estadísticos básicos de turnos y rating_diff:')
print(df[['turns','rating_diff']].describe())

In [ ]:
# Distribución del target y duración de batalla
fig, axes = plt.subplots(1, 2, figsize=(12,4))
sns.countplot(x='won_battle', data=df, ax=axes[0])
axes[0].set_title('Victorias por equipo (1=gana)')
sns.histplot(df['turns'], bins=30, ax=axes[1])
axes[1].set_title('Distribución de turnos por batalla')
plt.tight_layout()

In [ ]:
# Diferencia de rating vs probabilidad de victoria
bins = pd.cut(df['rating_diff'].fillna(0), bins=[-200,-100,0,100,200], include_lowest=True)
sns.barplot(x=bins, y=df['won_battle'], estimator=np.mean)
plt.xticks(rotation=45)
plt.ylabel('Win rate promedio')
plt.title('Impacto de rating_diff en la victoria')
plt.tight_layout()

In [ ]:
# Pokémon más frecuentes en los equipos
top_pokemon = (df['team_pokemon'].str.split(',').explode().value_counts().head(15))
plt.figure(figsize=(8,5))
sns.barplot(x=top_pokemon.values, y=top_pokemon.index)
plt.title('Pokémon más usados en Gen9 OU (dataset)')
plt.xlabel('Apariciones (sobre {} equipos)'.format(len(df)))
plt.tight_layout()

In [ ]:
# Matriz de correlación de estadísticas agregadas
stat_cols = [c for c in df.columns if c.startswith('sum_') or c.startswith('mean_')]
plt.figure(figsize=(10,8))
sns.heatmap(df[stat_cols + ['turns','rating_diff']].corr(), cmap='coolwarm', center=0)
plt.title('Correlaciones entre features numéricas originales')
plt.tight_layout()

## 3. Feature Engineering y preprocesamiento

* Features numéricas: sumas/promedios base, duración, rating, tamaño del equipo y métricas agregadas (ofensiva, bulk, spreads).
* Features categóricas: indicadores para los 40 Pokémon más frecuentes (sinergias básicas del metagame).
* Manejo de valores faltantes: se imputan `rating_diff` faltantes con 0 y se añade `has_rating_info`.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

df['rating_diff'] = df['rating_diff'].fillna(0)
df['team_size'] = df['team_size'].fillna(df['team_size'].median())
df['has_rating_info'] = df[['player_rating','opponent_rating']].notna().all(axis=1).astype(int)

df['agg_offense'] = df['sum_attack'] + df['sum_sp_attack']
df['agg_defense'] = df['sum_defense'] + df['sum_sp_defense']
df['bulk_index'] = df['sum_hp'] + df['sum_defense'] + df['sum_sp_defense']
df['offense_defense_diff'] = df['agg_offense'] - df['agg_defense']
df['speed_pressure'] = df['mean_speed'] * df['mean_attack']
df['stat_spread'] = df[stat_cols].std(axis=1)

feature_cols = stat_cols + ['turns','rating_diff','has_rating_info','team_size','agg_offense','agg_defense','bulk_index','offense_defense_diff','speed_pressure','stat_spread']

top40 = df['team_pokemon'].str.split(',').explode().value_counts().head(40).index.tolist()
mlb = MultiLabelBinarizer(classes=top40)
poke_matrix = mlb.fit_transform(df['team_pokemon'].str.split(','))
poke_cols = [f"pk_{name.lower().replace(' ','_').replace('-', '_')}" for name in mlb.classes_]
poke_df = pd.DataFrame(poke_matrix, columns=poke_cols, index=df.index)

feature_df = pd.concat([df[feature_cols], poke_df], axis=1)
X = feature_df
y = df['won_battle']
print('Shape de features:', X.shape)

## 4. Split Train/Test y Pipeline de preprocesamiento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

numeric_features = feature_cols
binary_features = poke_cols

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('bin', 'passthrough', binary_features)
    ]
)

print('Train:', X_train.shape, 'Test:', X_test.shape)

## 5. Comparación inicial de modelos (CV estratificada)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold

models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=300, random_state=42),
    'SVM-RBF': SVC(kernel='rbf', probability=True, C=2, gamma='scale', random_state=42),
    'LightGBM': LGBMClassifier(objective='binary', n_estimators=500, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9, random_state=42)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {'f1': 'f1', 'roc_auc': 'roc_auc', 'precision': 'precision', 'recall': 'recall'}

results = []
for name, model in models.items():
    pipe = Pipeline([('preprocess', preprocess), ('model', model)])
    scores = cross_validate(pipe, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1)
    results.append({
        'modelo': name,
        'f1_mean': scores['test_f1'].mean(),
        'roc_auc_mean': scores['test_roc_auc'].mean(),
        'precision_mean': scores['test_precision'].mean(),
        'recall_mean': scores['test_recall'].mean()
    })

cv_results = pd.DataFrame(results).sort_values('f1_mean', ascending=False)
cv_results

## 6. Búsqueda de hiperparámetros con validación cruzada

Se afinan los modelos con mejor desempeño (Random Forest y LightGBM) usando `GridSearchCV` estratificado.

In [ ]:
from sklearn.model_selection import GridSearchCV

rf_pipe = Pipeline([('preprocess', preprocess), ('model', RandomForestClassifier(random_state=42))])
rf_grid = {
    'model__n_estimators': [200, 400],
    'model__max_depth': [None, 15],
    'model__min_samples_split': [2, 5]
}
rf_search = GridSearchCV(rf_pipe, rf_grid, cv=cv, scoring='f1', n_jobs=-1)
rf_search.fit(X_train, y_train)
print('RF mejor F1:', rf_search.best_score_)
print('RF mejores params:', rf_search.best_params_)

lgb_pipe = Pipeline([('preprocess', preprocess), ('model', LGBMClassifier(objective='binary', random_state=42))])
lgb_grid = {
    'model__n_estimators': [300, 500],
    'model__learning_rate': [0.05, 0.1],
    'model__num_leaves': [31, 63],
    'model__subsample': [0.9],
    'model__colsample_bytree': [0.8, 1.0]
}
lgb_search = GridSearchCV(lgb_pipe, lgb_grid, cv=cv, scoring='f1', n_jobs=-1)
lgb_search.fit(X_train, y_train)
print('LightGBM mejor F1:', lgb_search.best_score_)
print('LightGBM mejores params:', lgb_search.best_params_)

## 7. Evaluación final en test hold-out

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay

best_model = lgb_search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot(cmap='Blues')
plt.show()

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.show()

## 8. Conclusiones y próximos pasos

* Dataset real balanceado: 828 equipos con etiquetas obtenidas de replays.
* Las estadísticas agregadas + indicadores de composiciones capturan parte de la señal; LightGBM optimizado logró el mejor F1 (~0.62 en CV).
* Random Forest queda como modelo visto en clase con buen rendimiento; Logistic Regression sirve como baseline reproducible.
* Próximos pasos: enriquecer features con roles/tipos, integrar info de objetos/movimientos y evaluar modelos específicos para sets (Set Transformers / Deep Sets).